In [16]:
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Optimizer

import scipy.io
from model import BayesLinear_Normalq, BBP_Model
from utils import log_gaussian_loss, gaussian, get_kl_Gaussian_divergence

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device: {}'.format(device))

device: cuda


In [17]:
class BBP_Model_PINN:
    def __init__(self, xt_lb, xt_ub, u_lb, u_ub,
                 input_dim, output_dim, no_units,
                 learn_rate, batch_size, no_batches,
                 prior_lambda1, prior_lambda2, num_epochs):

        
        self.xt_lb = torch.from_numpy(xt_lb).float().to(device)
        self.xt_ub = torch.from_numpy(xt_ub).float().to(device)
        self.u_lb = torch.from_numpy(u_lb).float().to(device)
        self.u_ub = torch.from_numpy(u_ub).float().to(device)


        self.learn_rate = learn_rate
        self.batch_size = batch_size
        self.no_batches = no_batches

        self.network = BBP_Model(input_dim = input_dim, output_dim = output_dim,
                                    no_units = no_units)


        self.prior_lambda1 = prior_lambda1
        self.prior_lambda2 = prior_lambda2

        self.lambda1_mus = nn.Parameter(torch.Tensor(1).uniform_(0, 2))
        self.lambda1_rhos = nn.Parameter(torch.Tensor(1).uniform_(-3, 2))
        self.lambda2_mus = nn.Parameter(torch.Tensor(1).uniform_(0, 0.05))
        self.lambda2_rhos = nn.Parameter(torch.Tensor(1).uniform_(-3, -2))

        self.network.register_parameter('lambda1_mu', self.lambda1_mus)
        self.network.register_parameter('lambda2_mu', self.lambda2_mus)
        self.network.register_parameter('lambda1_rho', self.lambda1_rhos)
        self.network.register_parameter('lambda2_rho', self.lambda2_rhos)

        self.network = self.network.to(device)

        # self.optimizer = torch.optim.SGD(self.network.parameters(), lr = self.learn_rate)
        self.optimizer = torch.optim.AdamW(self.network.parameters(), lr = self.learn_rate)
        # self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, max_lr = 1e-3, 
        #                                     steps_per_epoch = no_batches, epochs = num_epochs)
        self.loss_func = log_gaussian_loss

    def net_U(self, x, t):
        xt = torch.cat((x,t), dim=1)
        xt = 2*(xt-self.xt_lb)/(self.xt_ub-self.xt_lb) - 1
        out, KL_loss = self.network(xt)

        u = out[:,0:1]
        log_noise_u = out[:,1:2]
        
        return u, log_noise_u, KL_loss

    def net_F(self, x, t, lambda1_sample, lambda2_sample):
        lambda_1 = lambda1_sample        
        lambda_2 = torch.exp(lambda2_sample)

        u, _, _ = self.net_U(x, t)
        u = u*(self.u_ub-self.u_lb) + self.u_lb # reverse scaling
        # u = u*self.u_std + self.u_mean

        u_t = torch.autograd.grad(u, t, torch.ones_like(u),
                                    retain_graph=True,
                                    create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, torch.ones_like(u),
                                    retain_graph=True,
                                    create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, torch.ones_like(u_x),
                                    retain_graph=True,
                                    create_graph=True)[0]
        u_xxx = torch.autograd.grad(u_xx, x, torch.ones_like(u_xx),
                                    retain_graph=True,
                                    create_graph=True)[0]

        F = u_t + lambda_1*u*u_x + lambda_2*u_xxx
        return F

    def fit(self, X, t, U, no_samples):
        self.network.train()

        U = (U-self.u_lb)/(self.u_ub-self.u_lb) # scaling
      

        # reset gradient and total loss
        self.optimizer.zero_grad()

        fit_loss_total = 0
        fit_loss_F_total = 0
        fit_loss_U_total = 0

        for i in range(no_samples):
            lambda1_epsilons = self.lambda1_mus.data.new(self.lambda1_mus.size()).normal_()
            lambda1_stds = torch.log(1 + torch.exp(self.lambda1_rhos))
            lambda2_epsilons = self.lambda2_mus.data.new(self.lambda2_mus.size()).normal_()
            lambda2_stds = torch.log(1 + torch.exp(self.lambda2_rhos))

            lambda1_sample = self.lambda1_mus + lambda1_epsilons * lambda1_stds
            lambda2_sample = self.lambda2_mus + lambda2_epsilons * lambda2_stds

            u_pred, log_noise_u, KL_loss_para = self.net_U(X, t)
            f_pred = self.net_F(X, t, lambda1_sample, lambda2_sample)

            # calculate fit loss based on mean and standard deviation of output
            fit_loss_U_total += self.loss_func(u_pred, U, log_noise_u.exp(), self.network.output_dim)
            fit_loss_F_total += torch.sum(f_pred**2) ######
            # fit_loss_F_total += self.loss_func(f_pred, torch.zeros_like(f_pred), self.network.log_noise_f.exp(), self.network.output_dim)

        KL_loss_lambda1 = get_kl_Gaussian_divergence(self.prior_lambda1.mu, self.prior_lambda1.sigma**2, self.lambda1_mus, lambda1_stds**2)
        KL_loss_lambda2 = get_kl_Gaussian_divergence(self.prior_lambda2.mu, self.prior_lambda2.sigma**2, self.lambda2_mus, lambda2_stds**2)
        KL_loss_total = KL_loss_para + KL_loss_lambda1 + KL_loss_lambda2

        # minibatches and KL reweighting
        KL_loss_total = KL_loss_total/self.no_batches
        total_loss = (KL_loss_total + fit_loss_U_total + fit_loss_F_total) / (no_samples*X.shape[0])
        
        total_loss.backward()
        self.optimizer.step()
        # self.scheduler.step()

        return fit_loss_U_total/no_samples, fit_loss_F_total/no_samples, KL_loss_total, total_loss

    def predict(self, xt, no_sample, best_net):
        xt = torch.tensor(xt, requires_grad=True).float().to(device)
        xt = 2*(xt-self.xt_lb)/(self.xt_ub-self.xt_lb) - 1

        self.network.eval()
        samples = []
        noises = []
        for i in range(no_sample):
            out_pred, _ = best_net(xt)
            u_pred = out_pred[:,0:1]
            noise_u = out_pred[:,1:2].exp()

            u_pred = u_pred*(self.u_ub-self.u_lb) + self.u_lb # reverse scaling
            noise_u = noise_u*(self.u_ub-self.u_lb)

            samples.append(u_pred.detach().cpu().numpy())
            noises.append(noise_u.detach().cpu().numpy())
        return np.array(samples), np.array(noises)

    def sample_F(self, xt, no_samples = 100):
        xt = torch.tensor(xt, requires_grad=True).float().to(device)
        xt = 2*(xt-self.xt_lb)/(self.xt_ub-self.xt_lb) - 1
        self.network.eval()

        samples = []
        for _ in range(no_samples):
            lambda1_epsilons = self.lambda1_mus.data.new(self.lambda1_mus.size()).normal_()
            lambda1_stds = torch.log(1 + torch.exp(self.lambda1_rhos))
            lambda2_epsilons = self.lambda2_mus.data.new(self.lambda2_mus.size()).normal_()
            lambda2_stds = torch.log(1 + torch.exp(self.lambda2_rhos))

            lambda1_sample = self.lambda1_mus + lambda1_epsilons * lambda1_stds
            lambda2_sample = self.lambda2_mus + lambda2_epsilons * lambda2_stds
            f = self.net_F(xt[:,0:1], xt[:,1:2], lambda1_sample, lambda2_sample)
            samples.append(f.detach().cpu().numpy().ravel())
        return np.array(samples)



In [18]:
data = scipy.io.loadmat('./Data/KdV.mat')

t = data['tt'].flatten()[:,None] # 201 x 1
x = data['x'].flatten()[:,None] # 512 x 1
Exact_ = np.real(data['uu']).T # 201 x 512

noise = 0.1
Exact = Exact_ + noise*np.std(Exact_)*np.random.randn(201, 512)

X, T = np.meshgrid(x,t) # 201 x 512
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None])) # 102912 x 2
u_star = Exact.flatten()[:,None]  # 102912 x 1

# Domain bounds of x, t
xt_lb = X_star.min(0)
xt_ub = X_star.max(0)

# training data
N_u = 2000
idx = np.random.choice(X_star.shape[0], N_u, replace=False)
X_u_train = X_star[idx,:]

u_train = u_star[idx,:]

u_min = u_train.min(0)
u_max = u_train.max(0)

num_epochs, batch_size = 25000, len(X_u_train),

net = BBP_Model_PINN(xt_lb, xt_ub, u_min, u_max,
                     input_dim = 2, output_dim = 2, no_units = 50, learn_rate = 1e-3,
                        batch_size = batch_size, no_batches = 1,
                        prior_lambda1 = gaussian(0, 1), prior_lambda2 = gaussian(0, 1), num_epochs = num_epochs)


In [19]:
fit_loss_U_train = np.zeros(num_epochs)
fit_loss_F_train = np.zeros(num_epochs)
KL_loss_train = np.zeros(num_epochs)
loss = np.zeros(num_epochs)
noise = []


X = torch.tensor(X_u_train[:,0:1], requires_grad = True, device = device).float()
t = torch.tensor(X_u_train[:,1:2], requires_grad = True, device = device).float()
U = torch.tensor(u_train, requires_grad = True, device = device).float()

X_u_test_25 = np.hstack([x, 0.25*np.ones_like((x))])
target_25 = Exact_[50].reshape(-1,1)

In [20]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(comment = '_KdV')

for i in range(num_epochs):

    EU, EF, KL_loss, total_loss = net.fit(X, t, U, no_samples = 20)
    
    fit_loss_U_train[i] = EU.item()
    fit_loss_F_train[i] = EF.item()
    KL_loss_train[i] = KL_loss.item()
    loss[i] = total_loss.item()

    writer.add_scalar("loss/total_loss", loss[i], i)
    writer.add_scalar("loss/U_loss", fit_loss_U_train[i], i)
    writer.add_scalar("loss/F_loss", fit_loss_F_train[i], i)
    writer.add_scalar("loss/KL_loss", KL_loss_train[i], i)
    

    if i % 2000 == 0:
        F_test = net.sample_F(X_u_test_25)
        fig, axs = plt.subplots(2, 2, figsize=(20, 8))
        axs[0,0].hist(F_test[:,0])
        axs[0,1].hist(F_test[:,100])
        axs[1,0].hist(F_test[:,150])
        axs[1,1].hist(F_test[:,255])
        plt.savefig('./plots/kdv_epoch{}.tiff'.format(i))


    if i % 100 == 0 or i == num_epochs - 1:

        print("Epoch: {:5d}/{:5d}, total loss = {:.3f}, Fit loss U = {:.3f}, Fit loss F = {:.3f}, KL loss = {:.3f}".format(i + 1, num_epochs, 
               loss[i], fit_loss_U_train[i], fit_loss_F_train[i], KL_loss_train[i]))

    
        lambda1_mus = net.lambda1_mus.item()
        lambda1_stds = torch.log(1 + torch.exp(net.lambda1_rhos)).item()
        
        lambda2_mus = np.exp(net.lambda2_mus.item())
        lambda2_stds = torch.log(1 + torch.exp(net.lambda2_rhos)).item()
        
        samples_25, _ = net.predict(X_u_test_25, 100, net.network)
        u_pred_25 = samples_25.mean(axis=0)
        error_25 = np.linalg.norm(target_25-u_pred_25, 2)/np.linalg.norm(target_25, 2)

        samples_train, _ = net.predict(X_u_train, 100, net.network)
        u_pred_train = samples_train.mean(axis=0)
        error_train = np.linalg.norm(u_train-u_pred_train, 2)/np.linalg.norm(u_train, 2)


        # writer.add_scalars("loss/train_test", {'train':error_train, 'test':error_25}, i)
        # writer.add_scalars("loss/f_u", {'noise_f':noise_f, 'noise_u':noise_u}, i)
       
        print("Epoch: {:5d}/{:5d}, lambda1_mu = {:.3f}, lambda2_mu = {:.3f}, lambda1_std = {:.3f}, lambda2_std = {:.3f}".format(i + 1, num_epochs,
                                                                                                                        lambda1_mus, lambda2_mus,
                                                                                                                        lambda1_stds, lambda2_stds))
        print("Epoch: {:5d}/{:5d}, error_25 = {:.5f}, error_train = {:.5f}".format(i+1, num_epochs, error_25, error_train))
        # print("Epoch: {:5d}/{:5d}, noise_f = {:.5f}, noise_u = {:.5f}".format(i+1, num_epochs, noise_f, noise_u))
        print()

writer.close()

Epoch:     1/25000, total loss = 1.677, Fit loss U = 2020.866, Fit loss F = 5.173, KL loss = 26574.855
Epoch:     1/25000, lambda1_mu = 0.678, lambda2_mu = 1.043, lambda1_std = 1.789, lambda2_std = 0.091
Epoch:     1/25000, error_25 = 1.71311, error_train = 1.71122

Epoch:   101/25000, total loss = 0.798, Fit loss U = 276.270, Fit loss F = 5.607, KL loss = 26295.662
Epoch:   101/25000, lambda1_mu = 0.627, lambda2_mu = 1.030, lambda1_std = 1.727, lambda2_std = 0.100
Epoch:   101/25000, error_25 = 1.00004, error_train = 1.00264

Epoch:   201/25000, total loss = 0.732, Fit loss U = 148.342, Fit loss F = 7.551, KL loss = 26173.191
Epoch:   201/25000, lambda1_mu = 0.612, lambda2_mu = 1.025, lambda1_std = 1.706, lambda2_std = 0.110
Epoch:   201/25000, error_25 = 1.00823, error_train = 1.00035

Epoch:   301/25000, total loss = 0.798, Fit loss U = 290.179, Fit loss F = 3.261, KL loss = 26050.551
Epoch:   301/25000, lambda1_mu = 0.593, lambda2_mu = 1.017, lambda1_std = 1.684, lambda2_std = 0.12

In [ ]:

x = data['x'].flatten()[:,None]
X_u_test_25 = np.hstack([x, 0.25*np.ones_like((x))]); u_test_25 = Exact[50]; u_mean_25 = Exact_[50]
X_u_test_50 = np.hstack([x, 0.50*np.ones_like((x))]); u_test_50 = Exact[100]; u_mean_50 = Exact_[100]
X_u_test_75 = np.hstack([x, 0.75*np.ones_like((x))]); u_test_75 = Exact[150]; u_mean_75 = Exact_[150]


def get_res(X):
    samples, noises = net.predict(X, 100, net.network)
    u_pred = samples.mean(axis = 0)

    aleatoric = (noises**2).mean(axis = 0)**0.5
    epistemic = samples.var(axis = 0)**0.5
    total_unc = (aleatoric**2 + epistemic**2)**0.5
    return u_pred.ravel(), aleatoric.ravel(), epistemic.ravel(), total_unc.ravel()


x = x.ravel()
u_pred_25, ale_25, epi_25, total_unc_25 = get_res(X_u_test_25)
u_pred_50, ale_50, epi_50, total_unc_50 = get_res(X_u_test_50)
u_pred_75, ale_75, epi_75, total_unc_75 = get_res(X_u_test_75)



In [ ]:

fig, axs = plt.subplots(1, 3, figsize = (15,4))
axs[0].scatter(x, u_test_25, s = 10, marker = 'x', color = 'black', alpha = 0.5, label = 'Exact')
axs[0].plot(x, u_mean_25, 'b-', linewidth = 2, label = 'Prediction')
axs[0].plot(x, u_pred_25, 'r--', linewidth = 2, label = 'Prediction')
# axs[0].fill_between(x, u_pred_25-epi_25, u_pred_25+epi_25, color = 'g', alpha = 0.5, label = 'Epistemic + Aleatoric')
# axs[0].fill_between(x, u_pred_25-ale_25, u_pred_25+ale_25, color = 'g', alpha = 0.5, label = 'Epistemic + Aleatoric')
axs[0].fill_between(x, u_pred_25-2*total_unc_25, u_pred_25+2*total_unc_25, color = 'g', alpha = 0.5, label = 'Epistemic + Aleatoric')
axs[0].set_xlabel('$x$')
axs[0].set_ylabel('$u(t,x)$')
axs[0].set_title('$t = 0.25$', fontsize = 10)
# axs[0].set_xlim([-1.1,1.1])
# ax.set_ylim([-1.1,1.1])


axs[1].scatter(x, u_test_50, s = 10, marker = 'x', color = 'black', alpha = 0.5, label = 'Exact')
axs[1].plot(x, u_mean_50, 'b-', linewidth = 2, label = 'Prediction')
axs[1].plot(x, u_pred_50, 'r--', linewidth = 2, label = 'Prediction')
axs[1].fill_between(x, u_pred_50-2*total_unc_50, u_pred_50+2*total_unc_50, color = 'g', alpha = 0.5, label = 'Epistemic + Aleatoric')
axs[1].set_xlabel('$x$')
axs[1].set_ylabel('$u(t,x)$')
axs[1].set_title('$t = 0.5$', fontsize = 10)


axs[2].scatter(x, u_test_75, s = 10, marker = 'x', color = 'black', alpha = 0.5, label = 'Exact')
axs[2].plot(x, u_mean_75, 'b-', linewidth = 2, label = 'Prediction')
axs[2].plot(x, u_pred_75, 'r--', linewidth = 2, label = 'Prediction')
axs[2].fill_between(x, u_pred_75-2*total_unc_75, u_pred_75+2*total_unc_75, color = 'g', alpha = 0.5, label = 'Epistemic + Aleatoric')
axs[2].set_xlabel('$x$')
axs[2].set_ylabel('$u(t,x)$')
axs[2].set_title('$t = 0.75$', fontsize = 10)


plt.savefig('./plots/final_prediction_kdv.tiff')
